In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
# Define a single liquid neuron that updates its state based on inputs and previous state
class LiquidNeuron(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        """
        Initialize the LiquidNeuron with input and recurrent weights.
        Args:
        - input_dim: Dimension of input features
        - hidden_dim: Dimension of the hidden state
        """
        super(LiquidNeuron, self).__init__()
        self.hidden_dim = hidden_dim

        # Parameters for input-to-hidden connections
        self.weight_input = nn.Parameter(torch.randn(input_dim, hidden_dim))

        # Parameters for hidden-to-hidden (recurrent) connections
        self.weight_recurrent = nn.Parameter(torch.randn(hidden_dim, hidden_dim))

        # Bias term for the neuron
        self.bias = nn.Parameter(torch.zeros(hidden_dim))

    def forward(self, x, state):
        """
        Perform a forward pass for a single time step.
        Args:
        - x: Input at the current time step (batch_size x input_dim)
        - state: Previous hidden state (batch_size x hidden_dim)

        Returns:
        - new_state: Updated hidden state
        """
        # Compute the state update using a simplified differential equation
        state_update = torch.tanh(x @ self.weight_input + state @ self.weight_recurrent + self.bias)

        # Update the hidden state (simplified Euler integration)
        new_state = state + state_update

        return new_state



In [3]:
# Define the Liquid Neural Network model
class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Initialize the Liquid Neural Network.
        Args:
        - input_dim: Dimension of input features
        - hidden_dim: Dimension of the hidden state
        - output_dim: Dimension of the output
        """
        super(LiquidNeuralNetwork, self).__init__()
        self.hidden_dim = hidden_dim

        # The recurrent core consisting of liquid neurons
        self.liquid_neuron = LiquidNeuron(input_dim, hidden_dim)

        # Fully connected layer to map hidden state to output
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """
        Perform a forward pass through the network.
        Args:
        - x: Input tensor (batch_size x seq_len x input_dim)

        Returns:
        - output: Final output of the network (batch_size x output_dim)
        """
        batch_size, seq_len, input_dim = x.size()

        # Initialize the hidden state as zeros
        state = torch.zeros(batch_size, self.hidden_dim)

        # Process each time step in the sequence
        for t in range(seq_len):
            state = self.liquid_neuron(x[:, t, :], state)  # Update the state with each time step

        # Pass the final hidden state through the fully connected layer
        output = self.fc(state)
        return output



In [6]:
# Example Usage
if __name__ == "__main__":
    # Set input, hidden, and output dimensions
    input_dim = 10  # Number of input features (10-dimensional input at each timestep)
    hidden_dim = 20  # Size of the hidden layer (internal neuron state)
    output_dim = 1  # Size of the output layer (single prediction for each sequence)

    # Instantiate the Liquid Neural Network model
    model = LiquidNeuralNetwork(input_dim, hidden_dim, output_dim)

    # Define the loss function (Mean Squared Error for regression)
    criterion = nn.MSELoss()

    # Define the optimizer (Adam optimizer for efficient gradient updates)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Dummy training data (batch_size=32, sequence_length=5, input_dim=10)
    inputs = torch.randn(32, 5, input_dim)  # 32 sequences, each of length 5 with 10 features per time step
    targets = torch.randn(32, output_dim)  # Corresponding target values (one target per sequence)

    # Print the head (first few rows) of the input data and target
    print("Head of the Input Data (first sequence in the batch):")
    print(inputs[0])  # Print the first sequence in the batch (size: [5, 10])

    print("\nCorresponding Target (Expected output):")
    print(targets[0])  # Print the target for the first sequence

    # Training loop (only for a few epochs to demonstrate)
    for epoch in range(5):  # Limiting to 5 epochs for demonstration
        optimizer.zero_grad()  # Zero the gradients from the previous step

        # Forward pass: Compute model outputs
        outputs = model(inputs)  # Pass the input data through the network

        # Print the outputs (predictions of the model)
        print(f"\nEpoch {epoch+1} - Model's Output for the first sequence:")
        print(outputs[0])  # Print the model's prediction for the first sequence

        # Compute the loss between predicted outputs and target values
        loss = criterion(outputs, targets)  # MSE Loss (Mean Squared Error)

        # Backward pass: Compute gradients
        loss.backward()

        # Update model parameters using the optimizer
        optimizer.step()

        # Print the loss after each epoch
        print(f"Loss after epoch {epoch+1}: {loss.item():.4f}")

Head of the Input Data (first sequence in the batch):
tensor([[ 0.3266,  1.0728,  1.5481, -0.8662, -0.1960, -0.5100,  0.2629, -0.0767,
         -1.8771,  0.2020],
        [-0.3393, -1.2468, -0.1941,  0.5752,  0.6814, -0.0170,  0.4620,  0.5776,
          1.5289, -2.5243],
        [-0.4314, -0.1533, -0.6434, -1.8260, -0.5662, -2.0817, -1.1770,  1.5435,
          0.1898, -0.2435],
        [ 0.7885,  1.7689, -0.2101, -0.1192,  0.4306,  1.0471, -1.6873, -0.0904,
          0.1646, -1.5400],
        [ 2.1102,  0.3155,  0.3826, -1.3880,  0.7542, -0.4804,  0.1008,  0.4054,
          1.9530, -0.2530]])

Corresponding Target (Expected output):
tensor([-2.3060])

Epoch 1 - Model's Output for the first sequence:
tensor([0.4523], grad_fn=<SelectBackward0>)
Loss after epoch 1: 5.9513

Epoch 2 - Model's Output for the first sequence:
tensor([0.4460], grad_fn=<SelectBackward0>)
Loss after epoch 2: 5.7352

Epoch 3 - Model's Output for the first sequence:
tensor([0.4397], grad_fn=<SelectBackward0>)
Loss 